# Prepara ambiente spark

In [2]:
!pip install pyspark==2.4.5

In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

# Obtem dados

## PER/DCOMP Tabelas

In [5]:
import urllib.request
url = 'https://receita.economia.gov.br/orientacao/tributaria/restituicao-ressarcimento-reembolso-e-compensacao/perdcomp/perdcomp_tabelas.mdb/@@download/file/perdcomp_tabelas.mdb'
filename = 'perdcomp_tabelas.mdb'
urllib.request.urlretrieve(url, filename)

('perdcomp_tabelas.mdb', <http.client.HTTPMessage at 0x2184e41ea08>)

## CPNJ

### Obtem html

In [33]:
import urllib.request
import os
url = 'http://200.152.38.155/CNPJ'
filename = 'CNPJ.html'
urllib.request.urlretrieve(url, filename)

('CNPJ.html', <http.client.HTTPMessage at 0x20619d8f608>)

### Converte para string (formato aceito pelo beautifulSoup pra obter os hrefs)

In [34]:
import codecs
f=codecs.open("CNPJ.html", 'r')
html_string = f.read()
f.close()

### Deleta o html

In [36]:
import os
os.remove('CNPJ.html')

PermissionError: [WinError 32] O arquivo já está sendo usado por outro processo: 'CNPJ.html'

### Obtem os hrefs

In [37]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

hrefs = []
common_substring = 'DADOS_ABERTOS_CNPJ_'
for a in soup.find_all('a', href=True):
    this = a['href']
    if (common_substring in this):
        hrefs.append(this)

### Baixa os zips

In [39]:
for href in hrefs:
    url = 'http://200.152.38.155/CNPJ/' + href
    filename = href
    urllib.request.urlretrieve(url, filename)